In [ ]:

# Code from kaggle evaluation site
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [ ]:
# Inference is performed (roughly) as follows, ignoring details like how we manage multiple videos:
import tflite_runtime.interpreter as tflite
interpreter = tflite.Interpreter(model_path)

found_signatures = list(interpreter.get_signature_list().keys())

if REQUIRED_SIGNATURE not in found_signatures:
    raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner("serving_default")
output = prediction_fn(inputs=frames)
sign = np.argmax(output["outputs"])

https://www.kaggle.com/code/jvthunder/lstm-baseline-for-starters-sign-language/notebook

In this competition, we should submit the model itself.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers

In [ ]:
# save the model
converter = tf.lite.TFLiteConverter.from_keras_model(inference_model)
tflite_model = converter.convert()
model_path = "model.tflite"

# submit the model
with open(model_path, 'wb') as f:
    f.write(tflite_model)
!zip submission.zip $model_path

https://www.kaggle.com/competitions/asl-signs/discussion/390182

Also explains conversion from Keras to TFLite

In [ ]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

# Tensorflow - Loading data

https://www.kaggle.com/code/roberthatch/gislr-feature-data-on-the-shoulders/notebook

If I understand it correctly, the flow is as follow:

1. Convert x_ into x containing avg face, avg pose, lips, left hand, right hand for every frame, e.g. [23, 84, 3]
2. Pad x such that first dim is divisible by 3: [23, 84, 3] -> [24, 84, 3]
3. Split padded x into three parts, and for each part, compute mean and std over frames, concat and flatten: [8, 84, 3] -> [2, 84, 3] -> [1, 504]
4. Add flattened mean and std of x to features list, which now contains means and stds of the three parts + overall mean and std
5. Resize x to [15, 84, 3], flatten and append it to features list

* final shape of data: x (94477, 5796) and y (94477,)
* takes ca 15 minutes to compile whole dataset

In [1]:
%pip install tqdm
import os

import json
from tqdm import tqdm
import numpy as np
import pandas as pd

import tensorflow as tf

Note: you may need to restart the kernel to use updated packages.


In [16]:
# limit dataset for quick test
QUICK_TEST = False
QUICK_LIMIT = 100

In [17]:
#for Kaggle
# LANDMARK_FILES_DIR = "/kaggle/input/asl-signs/train_landmark_files"
# TRAIN_FILE = "/kaggle/input/asl-signs/train.csv"
# label_map = json.load(open("/kaggle/input/asl-signs/sign_to_prediction_index_map.json", "r"))

#for local notebook
LANDMARK_FILES_DIR = "../data/asl-signs/train_landmark_files"
TRAIN_FILE = "../data/asl-signs/train.csv"
label_map = json.load(open("../data/asl-signs/sign_to_prediction_index_map.json", "r"))

In [18]:
# code from kaggle competition to load test data
ROWS_PER_FRAME = 543
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [19]:
def right_hand_percentage(x):
    right = tf.gather(x, right_hand_landmarks, axis=1)
    left = tf.gather(x, left_hand_landmarks, axis=1)
    right_count = tf.reduce_sum(tf.where(tf.math.is_nan(right), tf.zeros_like(right), tf.ones_like(right)))
    left_count = tf.reduce_sum(tf.where(tf.math.is_nan(left), tf.zeros_like(left), tf.ones_like(left)))
    return right_count / (left_count+right_count)

## Configuration

In [20]:
NUM_FRAMES = 15
SEGMENTS = 3

LEFT_HAND_OFFSET = 468
POSE_OFFSET = LEFT_HAND_OFFSET+21
RIGHT_HAND_OFFSET = POSE_OFFSET+33

## average over the entire face, and the entire 'pose'
averaging_sets = [[0, 468], [POSE_OFFSET, 33]]

lip_landmarks = [61, 185, 40, 39, 37,  0, 267, 269, 270, 409,
                 291,146, 91,181, 84, 17, 314, 405, 321, 375, 
                 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 
                 95, 88, 178, 87, 14,317, 402, 318, 324, 308]
left_hand_landmarks = list(range(LEFT_HAND_OFFSET, LEFT_HAND_OFFSET+21))
right_hand_landmarks = list(range(RIGHT_HAND_OFFSET, RIGHT_HAND_OFFSET+21))

point_landmarks = [item for sublist in [lip_landmarks, left_hand_landmarks, right_hand_landmarks] for item in sublist]

LANDMARKS = len(point_landmarks) + len(averaging_sets)
print(LANDMARKS)
INPUT_SHAPE = (NUM_FRAMES,LANDMARKS*3)

84


### Helper Functions

In [21]:
def tf_nan_mean(x, axis=0):
    return tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), x), axis=axis) / tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), tf.ones_like(x)), axis=axis)

def tf_nan_std(x, axis=0):
    d = x - tf_nan_mean(x, axis=axis)
    return tf.math.sqrt(tf_nan_mean(d * d, axis=axis))

def flatten_means_and_stds(x, axis=0):
    # Get means and stds
    x_mean = tf_nan_mean(x, axis=0)
    x_std  = tf_nan_std(x,  axis=0)

    x_out = tf.concat([x_mean, x_std], axis=0)
    x_out = tf.reshape(x_out, (1, INPUT_SHAPE[1]*2))
    x_out = tf.where(tf.math.is_finite(x_out), x_out, tf.zeros_like(x_out))
    return x_out

## TensorFlow Feature Preprocessing Layer

In [22]:
class FeatureGen(tf.keras.layers.Layer):
    def __init__(self):
        super(FeatureGen, self).__init__()
    
    def call(self, x_in):
#         print(right_hand_percentage(x))
        x_list = [tf.expand_dims(tf_nan_mean(x_in[:, av_set[0]:av_set[0]+av_set[1], :], axis=1), axis=1) for av_set in averaging_sets]
        x_list.append(tf.gather(x_in, point_landmarks, axis=1))
        x = tf.concat(x_list, 1)

        x_padded = x
        for i in range(SEGMENTS):
            p0 = tf.where( ((tf.shape(x_padded)[0] % SEGMENTS) > 0) & ((i % 2) != 0) , 1, 0)
            p1 = tf.where( ((tf.shape(x_padded)[0] % SEGMENTS) > 0) & ((i % 2) == 0) , 1, 0)
            paddings = [[p0, p1], [0, 0], [0, 0]]
            x_padded = tf.pad(x_padded, paddings, mode="SYMMETRIC")
        x_list = tf.split(x_padded, SEGMENTS)
        x_list = [flatten_means_and_stds(_x, axis=0) for _x in x_list]

        x_list.append(flatten_means_and_stds(x, axis=0))
        
        ## Resize only dimension 0. Resize can't handle nan, so replace nan with that dimension's avg value to reduce impact.
        x = tf.image.resize(tf.where(tf.math.is_finite(x), x, tf_nan_mean(x, axis=0)), [NUM_FRAMES, LANDMARKS])
        x = tf.reshape(x, (1, INPUT_SHAPE[0]*INPUT_SHAPE[1]))
        x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
        x_list.append(x)
        x = tf.concat(x_list, axis=1)
        return x

feature_converter = FeatureGen()

## One tests symbolic tensor, the other tests real data.
print(feature_converter(tf.keras.Input((543, 3), dtype=tf.float32, name="inputs")))

#for kaggle
#feature_converter(load_relevant_data_subset(f'/kaggle/input/asl-signs/{pd.read_csv(TRAIN_FILE).path[1]}'))

#for local notebook
feature_converter(load_relevant_data_subset(f'../data/asl-signs/{pd.read_csv(TRAIN_FILE).path[1]}'))


KerasTensor(type_spec=TensorSpec(shape=(1, 5796), dtype=tf.float32, name=None), name='feature_gen_2/concat_5:0', description="created by layer 'feature_gen_2'")


<tf.Tensor: shape=(1, 5796), dtype=float32, numpy=
array([[ 5.7140142e-01,  4.6732509e-01, -2.4906856e-05, ...,
         3.8143307e-01,  8.0332047e-01, -1.0697230e-01]], dtype=float32)>

In [23]:
def convert_row(row, right_handed=True):
    #for kaggle
    #x = load_relevant_data_subset(os.path.join("/kaggle/input/asl-signs", row[1].path))
    #for local notebook
    x = load_relevant_data_subset(os.path.join("../data/asl-signs", row[1].path))
    
    x = feature_converter(tf.convert_to_tensor(x)).cpu().numpy()
    return x, row[1].label

right_handed_signer = [26734, 28656, 25571, 62590, 29302, 
                       49445, 53618, 18796,  4718,  2044, 
                       37779, 30680]
left_handed_signer  = [16069, 32319, 36257, 22343, 27610, 
                       61333, 34503, 55372, ]
both_hands_signer   = [37055, ]

messy = [29302, ]

def convert_and_save_data():
    df = pd.read_csv(TRAIN_FILE)
    df['label'] = df['sign'].map(label_map)
    total = df.shape[0]
    if QUICK_TEST:
        total = QUICK_LIMIT
    npdata = np.zeros((total, INPUT_SHAPE[0]*INPUT_SHAPE[1] + (SEGMENTS+1)*INPUT_SHAPE[1]*2))
    nplabels = np.zeros(total)
    for i, row in tqdm(enumerate(df.iterrows()), total=total):
        (x,y) = convert_row(row)
        npdata[i,:] = x
        nplabels[i] = y
        if QUICK_TEST and i == QUICK_LIMIT - 1:
            break
        
    #save as np file
    np.save("../data/feature_data.npy", npdata)
    np.save("../data/feature_labels.npy", nplabels)
        
convert_and_save_data()

100%|██████████| 94477/94477 [12:50<00:00, 122.67it/s]


In [24]:
#load data
X = np.load("../data/feature_data.npy")
y = np.load("../data/feature_labels.npy")

print(X.shape, y.shape)

print(X[0, :].shape, X[0, :])

(94477, 5796) (94477,)
(5796,) [ 5.18924236e-01  3.42620254e-01  1.48732506e-05 ...  6.35599792e-02
  5.70323110e-01 -1.20788895e-01]


## Rocket Classifier

In [27]:
!pip install sktime
from sklearn.model_selection import train_test_split
from sktime.classification.kernel_based import RocketClassifier


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [29]:
clf = RocketClassifier(num_kernels=500) 
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)


Note: you may need to restart the kernel to use updated packages.


Total count of used landmarks: 104
